In [1]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from patsy import dmatrices
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [3]:
xtrcovbs = pd.read_sas("xtrfxcovbs_withbmd.sas7bdat",format='sas7bdat')
xtrcovbs.columns = xtrcovbs.columns.str.lower()
xtrcovbs.columns

Index(['replicate', 't_xtrdate', 'r_xtrdate', 'nonewosteofx_enddate',
       'nonewhipfx_enddate', 'qid', 't_fl_fea', 't_ctpo_ec', 't_ttar',
       't_ctar', 't_tbar', 't_ttbmd', 't_ctbmd', 't_ctth', 't_tbbmd', 't_tbn',
       't_tbth', 't_tbsp', 't_moart', 'r_fl_fea', 'r_ctpo_ec', 'r_ttar',
       'r_ctar', 'r_tbar', 'r_ttbmd', 'r_ctbmd', 'r_ctth', 'r_tbbmd', 'r_tbn',
       'r_tbth', 'r_tbsp', 'r_moart', 't_bvtv', 'r_bvtv', 'fnewopfxd',
       'fnewhipfxd', 'newosteofx', 'firstnewosteofx_site',
       'firstnewosteofx_year', 'firstnewosteofx_month', 'firstnewosteofx_day',
       'newosteofx_trauma', 'newhipfx', 'firstnewhipfx_year',
       'firstnewhipfx_month', 'firstnewhipfx_day', 'newhipfx_trauma',
       'nonewosteofx_enddispo', 'nonewhipfx_enddispo', 'age', 'height',
       'weight', 'centre', 'parenthipfx', 'prevfx_any', 'cig_cur',
       'nbalc_week', 'rheu_ever', 'cortico_oral_reg', 'secop', 'nbfall_pastyr',
       'everlost10lbs', 'osteomed', 'i', 'prev_backmt', 'prev_hipmt'

In [4]:
with pd.option_context('display.max_rows', None,):
    print(xtrcovbs.isnull().sum())

replicate                    0
t_xtrdate                   42
r_xtrdate                   46
nonewosteofx_enddate         0
nonewhipfx_enddate           0
qid                          0
t_fl_fea                    35
t_ctpo_ec                   32
t_ttar                      37
t_ctar                      37
t_tbar                      37
t_ttbmd                     37
t_ctbmd                     37
t_ctth                      37
t_tbbmd                     37
t_tbn                       37
t_tbth                      37
t_tbsp                      37
t_moart                     45
r_fl_fea                    49
r_ctpo_ec                   44
r_ttar                      44
r_ctar                      44
r_tbar                      44
r_ttbmd                     44
r_ctbmd                     44
r_ctth                      44
r_tbbmd                     44
r_tbn                       44
r_tbth                      44
r_tbsp                      44
r_moart                     53
t_bvtv  

In [5]:
xtrcovbs.describe(include=np.number)

,replicate,t_fl_fea,t_ctpo_ec,t_ttar,t_ctar,t_tbar,t_ttbmd,t_ctbmd,t_ctth,t_tbbmd,...,status,r_ttop,t_ttop,r_tthip,t_tthip,numberhits,id,l14_16,neck_16,tothip_16
count,1231.0,1196.000000,1199.000000,1194.000000,1194.000000,1194.000000,1194.000000,1194.000000,1194.000000,1194.000000,...,1231.000000,1185.000000,1189.000000,1185.000000,1189.000000,1231.000000,1231.000000,1029.000000,1015.000000,1015.000000
mean,1.0,5383.988712,3.028795,686.022057,92.394473,581.372200,254.978326,773.763692,0.950364,158.390081,...,0.191714,36.205907,36.282590,37.903797,37.982338,2.756296,834.553209,0.998679,0.705895,0.851710
std,0.0,973.729253,4.896780,107.888330,25.454452,113.367475,52.810507,86.883444,0.287123,38.342168,...,0.499394,14.653595,14.729293,14.419366,14.477738,1.290644,356.105098,0.179075,0.115445,0.127472
min,1.0,2519.000000,0.021500,410.400000,0.000000,289.100000,11.600000,20.300000,0.000000,10.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,218.000000,0.554672,0.275064,0.405257
25%,1.0,4665.500000,0.075750,608.200000,77.617520,500.200000,217.900000,732.200000,0.770000,133.578400,...,0.000000,29.000000,29.000000,31.000000,31.000000,2.000000,527.500000,0.874187,0.630040,0.753133
50%,1.0,5308.600000,0.101400,681.710120,93.119995,580.413360,255.000000,777.000000,0.960000,157.743700,...,0.000000,34.000000,34.000000,34.000000,34.000000,3.000000,835.000000,0.982413,0.690406,0.849280
75%,1.0,5982.850000,6.450000,754.175000,108.800000,654.875000,286.700000,827.258300,1.122539,185.876500,...,0.000000,42.000000,43.000000,45.000000,45.000000,4.000000,1142.500000,1.091184,0.770702,0.932123
max,1.0,8463.000000,21.810000,1076.300000,162.704430,983.100000,418.900000,976.800000,1.830607,278.500000,...,2.000000,80.000000,80.000000,80.000000,80.000000,8.000000,1450.000000,1.857849,1.159085,1.295089


In [8]:
#Create a new dataset where missing values for the variables: 
  #t_BVTV, t_TbSp, and t_ttBMD are replaced by their respective means 

# Let's look at the full extent of the damage for individuals with missing values for
# 1)t_bvtv: 
with pd.option_context('display.max_rows', None,):
    print(xtrcovbs[xtrcovbs['t_bvtv'].isnull()])

      replicate t_xtrdate  r_xtrdate nonewosteofx_enddate  \
85          1.0       NaT 2012-06-11  2015-06-15 00:00:00   
354         1.0       NaT 2013-06-13  2015-03-26 00:00:00   
395         1.0       NaT 2013-04-18  2015-07-10 00:00:00   
396         1.0       NaT 2013-04-18  2015-07-10 00:00:00   
462         1.0       NaT 2013-08-01  2015-12-14 00:00:00   
463         1.0       NaT 2013-08-01  2015-12-14 00:00:00   
464         1.0       NaT 2013-08-01  2015-12-14 00:00:00   
490         1.0       NaT 2013-07-23  2015-12-18 00:00:00   
560         1.0       NaT        NaT  2015-03-21 00:00:00   
561         1.0       NaT        NaT  2015-03-21 00:00:00   
612         1.0       NaT        NaT  2016-05-01 00:00:00   
613         1.0       NaT        NaT  2016-05-01 00:00:00   
627         1.0       NaT 2013-05-03  2016-04-28 00:00:00   
756         1.0       NaT        NaT  2015-06-02 00:00:00   
757         1.0       NaT        NaT  2015-06-02 00:00:00   
758         1.0       Na

In [10]:
# Create variable to define if missing or not:
xtrcovbs2 = xtrcovbs.copy()
xtrcovbs2['tbv_miss'] = xtrcovbs['t_bvtv'].apply(lambda x: 0 if pd.isnull(x) else 1)
np.unique(xtrcovbs2['tbv_miss'], return_counts=True)

(array([0, 1], dtype=int64), array([  37, 1194], dtype=int64))

In [11]:
# now model NON-missingness
xtrnmiss = xtrcovbs2[["id","tbv_miss","age","weight","height","prevfx_any","centre"]].dropna()
np.isnan(xtrnmiss).value_counts()

id     tbv_miss  age    weight  height  prevfx_any  centre
False  False     False  False   False   False       False     1229
dtype: int64

In [12]:
x = xtrnmiss[["age","weight","height","prevfx_any","centre"]] # 2D array
y = xtrnmiss["tbv_miss"]
x1 = sm.add_constant(x)

nmissmod = sm.Logit(y, x1).fit()
nmissmod.summary()

Optimization terminated successfully.
         Current function value: 0.120132
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               tbv_miss   No. Observations:                 1229
Model:                          Logit   Df Residuals:                     1223
Method:                           MLE   Df Model:                            5
Date:                Mon, 11 Apr 2022   Pseudo R-squ.:                  0.1109
Time:                        22:56:17   Log-Likelihood:                -147.64
converged:                       True   LL-Null:                       -166.05
Covariance Type:            nonrobust   LLR p-value:                 6.524e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.3665      5.353     -2.310      0.021     -22.858      -1.875
age            0.0824      0.020      4.036      0.000       0.042       0.122
weight        -0.0308      0.010     -3.053      0.002      -0.051      -0.011
height         0.0860      0.031      2.768      0.006       0.025       0.147
prevfx_any    -0.8203      0.349     -2.353      0.019      -1.504      -0.137
centre        -0.3299      0.125     -2.638      0.008      -0.575      -0.085
==============================================================================
"""

In [13]:
# Generate predicted probabilities: 
xtrnmiss["prob"] = nmissmod.predict(x1)
xtrnmiss["ipw"] = 1/xtrnmiss["prob"]
xtrnmiss["ipw"].describe()

count    1229.000000
mean        1.032767
std         0.046035
min         1.000611
25%         1.010860
50%         1.019155
75%         1.035674
max         1.611027
Name: ipw, dtype: float64

In [15]:
# Now assign weights to individuals with missing data 
# Those without missing data will get a weight of 1 ie. not weighted up or down

xtrnmiss.loc[xtrnmiss['tbv_miss'] == 1, 'wt'] = 1
xtrnmiss.loc[xtrnmiss['tbv_miss'] == 0, 'wt'] = xtrnmiss['ipw']

In [16]:
np.unique(xtrnmiss['wt'])

array([1.        , 1.01305162, 1.01364313, 1.01407848, 1.01661939,
       1.02008814, 1.02749902, 1.0347812 , 1.04845569, 1.05376274,
       1.07056631, 1.07292269, 1.09958706, 1.1161447 , 1.11700021,
       1.22503103, 1.30864744, 1.61102723])

In [20]:
xtrnmiss['tbv_miss'].fillna(int(xtrnmiss['tbv_miss'].mean()))

0       1
1       1
2       1
3       1
4       1
       ..
1226    1
1227    1
1228    1
1229    1
1230    1
Name: tbv_miss, Length: 1229, dtype: int64

In [17]:
# Now apply it to a model: 

# Let's look at odds for new fractures - binary logistic regression 
# BUT remmeber - we have to get rid of missing values as the first step of our modeling
# We don't want to get rid of our t_fl_fea that are missing.... 
# Let's deal with missing values first 

# Next section


In [23]:
#Create a new dataset where missing values for the variables:   
#t_BVTV, t_TbSp, and t_ttBMD are replaced by their respective means 

xtrcovbsnm = xtrcovbs.copy() # create a copy# 

#rep[ace t_bvtv variable 
xtrcovbsnm['t_bvtv'] = xtrcovbsnm['t_bvtv'].replace(to_replace=np.nan, value=np.mean(xtrcovbsnm['t_bvtv']))
xtrcovbsnm['t_tbsp'] = xtrcovbsnm['t_tbsp'].replace(to_replace=np.nan, value=np.mean(xtrcovbsnm['t_tbsp']))
xtrcovbsnm['t_ttbmd'] = xtrcovbsnm['t_ttbmd'].replace(to_replace=np.nan, value=np.mean(xtrcovbsnm['t_ttbmd']))

In [27]:
#Create a new dataset where missing values for the variables: 
#t_BVTV, t_TbSp, and t_ttBMD are imputed by using a multivariable linear regression model that
#includes age, height, weight, prevfx_any, centre, l14_16, neck_16

# So let's take a look first: 

varsofinterest = ['r_fl_fea',"t_bvtv","t_tbsp",'t_ttbmd',"age","weight","height","prevfx_any","l14_16", "neck_16"]
xtrcovbs[varsofinterest][pd.isnull(xtrcovbs['t_fl_fea'])==True]

,r_fl_fea,t_bvtv,t_tbsp,t_ttbmd,age,weight,height,prevfx_any,l14_16,neck_16
85,2416.6,NaN,NaN,NaN,66.0,147.7,160.5,1.0,1.173846,0.731651
89,NaN,11.0070,0.5353,182.7496,81.0,76.1,157.0,1.0,0.755504,0.537680
90,NaN,11.0070,0.5353,182.7496,81.0,76.1,157.0,1.0,0.755504,0.537680
91,NaN,11.0070,0.5353,182.7496,81.0,76.1,157.0,1.0,0.755504,0.537680
195,NaN,10.1538,0.7041,221.6592,77.0,79.2,167.0,1.0,0.732540,0.593217
196,NaN,10.1538,0.7041,221.6592,77.0,79.2,167.0,1.0,0.732540,0.593217
560,NaN,NaN,NaN,NaN,87.0,68.0,156.3,1.0,NaN,NaN
561,NaN,NaN,NaN,NaN,87.0,68.0,156.3,1.0,NaN,NaN
612,NaN,NaN,NaN,NaN,82.0,84.0,156.4,0.0,0.988572,0.869211
613,NaN,NaN,NaN,NaN,82.0,84.0,156.4,0.0,0.988572,0.869211


In [29]:
# bad idea to include r_fl_fea 
xtrwt = xtrcovbs[["t_bvtv","t_tbsp",'t_ttbmd',"age","weight","height","prevfx_any","l14_16", "neck_16"]].dropna()
np.isnan(xtrwt).value_counts()

x = xtrwt[["age","weight","height","prevfx_any","l14_16", "neck_16"]] # 2D array
y = xtrwt["t_bvtv"]
x1 = sm.add_constant(x)

t_bvmodel = sm.OLS(y, x1).fit()
print(t_bvmodel.summary())

# Array containing all regression coefficients
t_bvmodel.params

                            OLS Regression Results                            
Dep. Variable:                 t_bvtv   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     9.507
Date:                Tue, 12 Apr 2022   Prob (F-statistic):           3.60e-10
Time:                        18:54:47   Log-Likelihood:                -3206.2
No. Observations:                 977   AIC:                             6426.
Df Residuals:                     970   BIC:                             6461.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6850      6.307      0.584      0.5

const         3.684983
age           0.094483
weight        0.079257
height       -0.045325
prevfx_any    0.623335
l14_16       -8.525340
neck_16       5.634371
dtype: float64

In [30]:
# show variables again
varsofinterest = ["t_bvtv","age","weight","height","prevfx_any","l14_16", "neck_16"]
xtrcovbs[varsofinterest][pd.isnull(xtrcovbs['t_bvtv'])==True]

,t_bvtv,age,weight,height,prevfx_any,l14_16,neck_16
85,NaN,66.0,147.7,160.5,1.0,1.173846,0.731651
354,NaN,85.0,60.0,151.0,1.0,0.743799,0.548450
395,NaN,68.0,73.0,156.0,1.0,0.834076,0.584830
396,NaN,68.0,73.0,156.0,1.0,0.834076,0.584830
462,NaN,60.0,51.0,157.0,1.0,0.828360,0.639716
463,NaN,60.0,51.0,157.0,1.0,0.828360,0.639716
464,NaN,60.0,51.0,157.0,1.0,0.828360,0.639716
490,NaN,67.0,73.0,148.0,1.0,0.998716,0.777012
560,NaN,87.0,68.0,156.3,1.0,NaN,NaN
561,NaN,87.0,68.0,156.3,1.0,NaN,NaN


In [31]:
# not too bad - only some are missing DXA L14 and Femoral Neck BMD: (L14_16 and Neck_16)
# We could replace these with mean values (this is fine since we are just estimating the best version of t_fl_fea

xtrcovbs['l14_16'] = xtrcovbs['l14_16'].replace(to_replace=np.nan, value=np.mean(xtrcovbs['l14_16']))
xtrcovbs['neck_16'] = xtrcovbs['neck_16'].replace(to_replace=np.nan, value=np.mean(xtrcovbs['neck_16']))

In [32]:
t_bvmodel.params

const         3.684983
age           0.094483
weight        0.079257
height       -0.045325
prevfx_any    0.623335
l14_16       -8.525340
neck_16       5.634371
dtype: float64

In [33]:
# Let's merge the list of coefficients together with the main dataframe 

regestdf = pd.DataFrame(t_bvmodel.params).T
regestdf2 = regestdf.add_prefix("b_")

# add dummy key column to merge one to many 
regestdf2['dumid']="key"
regestdf2

,b_const,b_age,b_weight,b_height,b_prevfx_any,b_l14_16,b_neck_16,dumid
0,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,key


In [35]:
# also add dummy key column to xtrcovbs
xtrcovbs['dumid']="key"
varsofinterest = ["t_bvtv","age","weight","height","prevfx_any","l14_16", "neck_16"]
varsofinterest2 = ["b_" + sub for sub in varsofinterest[1:]]
varsofinterest2.insert(0,"b_const")
varsofinterest2.extend(varsofinterest)

mainregestdf = pd.merge(xtrcovbs, regestdf2, on='dumid')
mainregestdf.drop('dumid', axis=1) # delete dummy id
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_bvtv'])==True]

,b_const,b_age,b_weight,b_height,b_prevfx_any,b_l14_16,b_neck_16,t_bvtv,age,weight,height,prevfx_any,l14_16,neck_16
85,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,66.0,147.7,160.5,1.0,1.173846,0.731651
354,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,85.0,60.0,151.0,1.0,0.743799,0.548450
395,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,68.0,73.0,156.0,1.0,0.834076,0.584830
396,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,68.0,73.0,156.0,1.0,0.834076,0.584830
462,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,60.0,51.0,157.0,1.0,0.828360,0.639716
463,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,60.0,51.0,157.0,1.0,0.828360,0.639716
464,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,60.0,51.0,157.0,1.0,0.828360,0.639716
490,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,67.0,73.0,148.0,1.0,0.998716,0.777012
560,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,87.0,68.0,156.3,1.0,0.998679,0.705895
561,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,NaN,87.0,68.0,156.3,1.0,0.998679,0.705895


In [37]:
# Now we could do our math: 
for i in mainregestdf.index:
    if pd.isnull(mainregestdf['t_bvtv'][i]):
        mainregestdf['t_bvtv'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]
    else: 
        mainregestdf['t_bvtv'][i] = mainregestdf['t_bvtv'][i]

<ipython-input-37-7cf9429d5fa6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_bvtv'][i] = mainregestdf['t_bvtv'][i]
<ipython-input-37-7cf9429d5fa6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_bvtv'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]


In [38]:
# see now that the missing values are filled in: 
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_bvtv'])==True]

,b_const,b_age,b_weight,b_height,b_prevfx_any,b_l14_16,b_neck_16,t_bvtv,age,weight,height,prevfx_any,l14_16,neck_16
85,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,9.090821,66.0,147.7,160.5,1.0,1.173846,0.731651
354,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,6.999794,85.0,60.0,151.0,1.0,0.743799,0.548450
395,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,5.632642,68.0,73.0,156.0,1.0,0.834076,0.584830
396,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,5.632642,68.0,73.0,156.0,1.0,0.834076,0.584830
462,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,3.445769,60.0,51.0,157.0,1.0,0.828360,0.639716
463,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,3.445769,60.0,51.0,157.0,1.0,0.828360,0.639716
464,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,3.445769,60.0,51.0,157.0,1.0,0.828360,0.639716
490,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,5.579968,67.0,73.0,148.0,1.0,0.998716,0.777012
560,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,6.296764,87.0,68.0,156.3,1.0,0.998679,0.705895
561,3.684983,0.094483,0.079257,-0.045325,0.623335,-8.52534,5.634371,6.296764,87.0,68.0,156.3,1.0,0.998679,0.705895


In [39]:
#t_tbsp


x = xtrwt[["age","weight","height","prevfx_any","l14_16", "neck_16"]] # 2D array
y = xtrwt["t_tbsp"]
x1 = sm.add_constant(x)

t_bvmodel = sm.OLS(y, x1).fit()
print(t_bvmodel.summary())

# Array containing all regression coefficients
t_bvmodel.params



# Let's merge the list of coefficients together with the main dataframe 

regestdf = pd.DataFrame(t_bvmodel.params).T
regestdf2 = regestdf.add_prefix("b_")

# add dummy key column to merge one to many 
regestdf2['dumid']="key"
regestdf2


# also add dummy key column to xtrcovbs
xtrcovbs['dumid']="key"
varsofinterest = ["t_tbsp","age","weight","height","prevfx_any","l14_16", "neck_16"]
varsofinterest2 = ["b_" + sub for sub in varsofinterest[1:]]
varsofinterest2.insert(0,"b_const")
varsofinterest2.extend(varsofinterest)

mainregestdf = pd.merge(xtrcovbs, regestdf2, on='dumid')
mainregestdf.drop('dumid', axis=1) # delete dummy id
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_bvtv'])==True]
# Now we could do our math: 
for i in mainregestdf.index:
    if pd.isnull(mainregestdf['t_tbsp'][i]):
        mainregestdf['t_tbsp'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]
    else: 
        mainregestdf['t_tbsp'][i] = mainregestdf['t_tbsp'][i]
# see now that the missing values are filled in: 
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_tbsp'])==True]

                            OLS Regression Results                            
Dep. Variable:                 t_tbsp   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     42.48
Date:                Tue, 12 Apr 2022   Prob (F-statistic):           3.78e-46
Time:                        19:05:25   Log-Likelihood:                 518.09
No. Observations:                 977   AIC:                            -1022.
Df Residuals:                     970   BIC:                            -988.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2009      0.139      8.614      0.0

<ipython-input-39-d3f75142f31d>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_tbsp'][i] = mainregestdf['t_tbsp'][i]
<ipython-input-39-d3f75142f31d>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_tbsp'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]


,b_const,b_age,b_weight,b_height,b_prevfx_any,b_l14_16,b_neck_16,t_tbsp,age,weight,height,prevfx_any,l14_16,neck_16
85,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.387968,66.0,147.7,160.5,1.0,1.173846,0.731651
354,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.642698,85.0,60.0,151.0,1.0,0.743799,0.548450
395,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.610694,68.0,73.0,156.0,1.0,0.834076,0.584830
396,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.610694,68.0,73.0,156.0,1.0,0.834076,0.584830
462,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.638311,60.0,51.0,157.0,1.0,0.828360,0.639716
463,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.638311,60.0,51.0,157.0,1.0,0.828360,0.639716
464,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.638311,60.0,51.0,157.0,1.0,0.828360,0.639716
490,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.531905,67.0,73.0,148.0,1.0,0.998716,0.777012
560,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.536624,87.0,68.0,156.3,1.0,0.998679,0.705895
561,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.536624,87.0,68.0,156.3,1.0,0.998679,0.705895


In [40]:
#t_tbsp


x = xtrwt[["age","weight","height","prevfx_any","l14_16", "neck_16"]] # 2D array
y = xtrwt["t_tbsp"]
x1 = sm.add_constant(x)

t_bvmodel = sm.OLS(y, x1).fit()
print(t_bvmodel.summary())

# Array containing all regression coefficients
t_bvmodel.params



# Let's merge the list of coefficients together with the main dataframe 

regestdf = pd.DataFrame(t_bvmodel.params).T
regestdf2 = regestdf.add_prefix("b_")

# add dummy key column to merge one to many 
regestdf2['dumid']="key"
regestdf2


# also add dummy key column to xtrcovbs
xtrcovbs['dumid']="key"
varsofinterest = ["t_tbsp","age","weight","height","prevfx_any","l14_16", "neck_16"]
varsofinterest2 = ["b_" + sub for sub in varsofinterest[1:]]
varsofinterest2.insert(0,"b_const")
varsofinterest2.extend(varsofinterest)

mainregestdf = pd.merge(xtrcovbs, regestdf2, on='dumid')
mainregestdf.drop('dumid', axis=1) # delete dummy id
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_tbsp'])==True]
# Now we could do our math: 
for i in mainregestdf.index:
    if pd.isnull(mainregestdf['t_tbsp'][i]):
        mainregestdf['t_tbsp'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]
    else: 
        mainregestdf['t_tbsp'][i] = mainregestdf['t_tbsp'][i]
# see now that the missing values are filled in: 
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_tbsp'])==True]

                            OLS Regression Results                            
Dep. Variable:                 t_tbsp   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     42.48
Date:                Tue, 12 Apr 2022   Prob (F-statistic):           3.78e-46
Time:                        19:07:51   Log-Likelihood:                 518.09
No. Observations:                 977   AIC:                            -1022.
Df Residuals:                     970   BIC:                            -988.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2009      0.139      8.614      0.0

<ipython-input-40-0a32be5362b9>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_tbsp'][i] = mainregestdf['t_tbsp'][i]
<ipython-input-40-0a32be5362b9>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_tbsp'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]


,b_const,b_age,b_weight,b_height,b_prevfx_any,b_l14_16,b_neck_16,t_tbsp,age,weight,height,prevfx_any,l14_16,neck_16
85,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.387968,66.0,147.7,160.5,1.0,1.173846,0.731651
354,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.642698,85.0,60.0,151.0,1.0,0.743799,0.548450
395,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.610694,68.0,73.0,156.0,1.0,0.834076,0.584830
396,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.610694,68.0,73.0,156.0,1.0,0.834076,0.584830
462,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.638311,60.0,51.0,157.0,1.0,0.828360,0.639716
463,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.638311,60.0,51.0,157.0,1.0,0.828360,0.639716
464,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.638311,60.0,51.0,157.0,1.0,0.828360,0.639716
490,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.531905,67.0,73.0,148.0,1.0,0.998716,0.777012
560,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.536624,87.0,68.0,156.3,1.0,0.998679,0.705895
561,1.20091,-0.001066,-0.001711,-0.000784,0.034732,-0.129477,-0.337241,0.536624,87.0,68.0,156.3,1.0,0.998679,0.705895


In [41]:
#t_ttbmd

x = xtrwt[["age","weight","height","prevfx_any","l14_16", "neck_16"]] # 2D array
y = xtrwt["t_ttbmd"]
x1 = sm.add_constant(x)

t_bvmodel = sm.OLS(y, x1).fit()
print(t_bvmodel.summary())

# Array containing all regression coefficients
t_bvmodel.params



# Let's merge the list of coefficients together with the main dataframe 

regestdf = pd.DataFrame(t_bvmodel.params).T
regestdf2 = regestdf.add_prefix("b_")

# add dummy key column to merge one to many 
regestdf2['dumid']="key"
regestdf2


# also add dummy key column to xtrcovbs
xtrcovbs['dumid']="key"
varsofinterest = ["t_ttbmd","age","weight","height","prevfx_any","l14_16", "neck_16"]
varsofinterest2 = ["b_" + sub for sub in varsofinterest[1:]]
varsofinterest2.insert(0,"b_const")
varsofinterest2.extend(varsofinterest)

mainregestdf = pd.merge(xtrcovbs, regestdf2, on='dumid')
mainregestdf.drop('dumid', axis=1) # delete dummy id
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_ttbmd'])==True]
# Now we could do our math: 
for i in mainregestdf.index:
    if pd.isnull(mainregestdf['t_ttbmd'][i]):
        mainregestdf['t_ttbmd'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]
    else: 
        mainregestdf['t_ttbmd'][i] = mainregestdf['t_ttbmd'][i]
# see now that the missing values are filled in: 
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_ttbmd'])==True]

                            OLS Regression Results                            
Dep. Variable:                t_ttbmd   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     125.6
Date:                Tue, 12 Apr 2022   Prob (F-statistic):          1.85e-117
Time:                        19:08:07   Log-Likelihood:                -4975.7
No. Observations:                 977   AIC:                             9965.
Df Residuals:                     970   BIC:                         1.000e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        621.7570     38.585     16.114      0.0

<ipython-input-41-4590b955d50b>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_ttbmd'][i] = mainregestdf['t_ttbmd'][i]
<ipython-input-41-4590b955d50b>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainregestdf['t_ttbmd'][i] = mainregestdf['b_age'][i]*mainregestdf['age'][i] + mainregestdf['b_weight'][i]*mainregestdf['weight'][i] + mainregestdf['b_height'][i]*mainregestdf['height'][i] + mainregestdf['b_prevfx_any'][i]*mainregestdf['prevfx_any'][i] + mainregestdf['b_l14_16'][i]*mainregestdf['l14_16'][i] + mainregestdf['b_neck_16'][i]*mainregestdf['neck_16'][i] + mainregestdf['b_const'][i]


,b_const,b_age,b_weight,b_height,b_prevfx_any,b_l14_16,b_neck_16,t_ttbmd,age,weight,height,prevfx_any,l14_16,neck_16
85,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,289.702060,66.0,147.7,160.5,1.0,1.173846,0.731651
354,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,208.745425,85.0,60.0,151.0,1.0,0.743799,0.548450
395,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,234.833017,68.0,73.0,156.0,1.0,0.834076,0.584830
396,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,234.833017,68.0,73.0,156.0,1.0,0.834076,0.584830
462,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,250.030195,60.0,51.0,157.0,1.0,0.828360,0.639716
463,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,250.030195,60.0,51.0,157.0,1.0,0.828360,0.639716
464,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,250.030195,60.0,51.0,157.0,1.0,0.828360,0.639716
490,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,307.762257,67.0,73.0,148.0,1.0,0.998716,0.777012
560,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,235.420178,87.0,68.0,156.3,1.0,0.998679,0.705895
561,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,235.420178,87.0,68.0,156.3,1.0,0.998679,0.705895


In [43]:
# see now that the missing values are filled in: 
mainregestdf[varsofinterest2][pd.isnull(xtrcovbs['t_bvtv'])==True]

,b_const,b_age,b_weight,b_height,b_prevfx_any,b_l14_16,b_neck_16,t_ttbmd,age,weight,height,prevfx_any,l14_16,neck_16
85,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,289.702060,66.0,147.7,160.5,1.0,1.173846,0.731651
354,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,208.745425,85.0,60.0,151.0,1.0,0.743799,0.548450
395,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,234.833017,68.0,73.0,156.0,1.0,0.834076,0.584830
396,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,234.833017,68.0,73.0,156.0,1.0,0.834076,0.584830
462,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,250.030195,60.0,51.0,157.0,1.0,0.828360,0.639716
463,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,250.030195,60.0,51.0,157.0,1.0,0.828360,0.639716
464,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,250.030195,60.0,51.0,157.0,1.0,0.828360,0.639716
490,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,307.762257,67.0,73.0,148.0,1.0,0.998716,0.777012
560,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,235.420178,87.0,68.0,156.3,1.0,0.998679,0.705895
561,621.757004,-1.526688,0.301778,-2.881169,-11.066167,24.75711,230.392236,235.420178,87.0,68.0,156.3,1.0,0.998679,0.705895


In [ ]:
#Use regression coefficients from this model along with intercepts to compute what the missing values should be

In [ ]:
#Run a binary logistic regression model looking at newosteofx as outcome, 
#t_TbSp as exposure, adjusting for age, height, weight, prevfx_any, secop, and osteomed:  

#Using original dataset with missing values 

#The dataset with missing values imputed using means

#The dataset with missing values imputed using regression model 

In [24]:
#Compare the differences in the odds ratios for t_TbSp from each of these datasets.